In [1]:
import pyaniasetools as pya
from rdkit import Chem
from rdkit.Chem import AllChem
import hdnntools as hdt
import numpy as np

van Der Waals correction will be unavailable. Please install ased3


In [2]:
netdir = '/scratch/Gits/ANI-Networks/networks/al_networks/ANI-AL-0808.0303.0400/'
cns = netdir+'train0/rHCNOSFCl-4.6A_16-3.1A_a4-8.params'
sae = netdir+'train0/sae_wb97x-631gd.dat'
nnf = netdir+'train'
Nn = 5

In [3]:
cmp = pya.anienscomputetool(cns,sae,nnf,Nn,0)

In [4]:
m = Chem.MolFromSmiles('COC1=CC=CC(=C1)N1CCN(CC1)C1=NC2=C(C=CC=C2F)[C@H](CC(O)=O)N1C1=CC(=CC=C1OC)C(F)(F)F')
m2=Chem.AddHs(m)
# run distance geometry 10 times
cids = AllChem.EmbedMultipleConfs(m2, useExpTorsionAnglePrefs=True, useBasicKnowledge=True, numConfs=50)
print(len(cids))
for cid in cids:
    print('Confid:',cid)
    _ = AllChem.MMFFOptimizeMolecule(m2, confId=cid)
    _ = cmp.optimize_rdkit_molecule(m2, cid=cid, fmax=0.001)
rmslist = []
AllChem.AlignMolConformers(m2, RMSlist=rmslist)
print(rmslist)

50
Confid: 0
Confid: 1
Confid: 2
Confid: 3
Confid: 4
Confid: 5
Confid: 6
Confid: 7
Confid: 8
Confid: 9
Confid: 10
Confid: 11
Confid: 12
Confid: 13
Confid: 14
Confid: 15
Confid: 16
Confid: 17
Confid: 18
Confid: 19
Confid: 20
Confid: 21
Confid: 22
Confid: 23
Confid: 24
Confid: 25
Confid: 26
Confid: 27
Confid: 28
Confid: 29
Confid: 30
Confid: 31
Confid: 32
Confid: 33
Confid: 34
Confid: 35
Confid: 36
Confid: 37
Confid: 38
Confid: 39
Confid: 40
Confid: 41
Confid: 42
Confid: 43
Confid: 44
Confid: 45
Confid: 46
Confid: 47
Confid: 48
Confid: 49
[2.346998358221117, 3.882753298387711, 3.4135364893479574, 3.1240679558105247, 3.705360247974471, 3.9279631444220175, 3.0325541352388186, 2.644137280140896, 3.6856467739004666, 2.124798967684887, 2.9123091072508296, 3.64328618149378, 3.1627332038397964, 2.8360422338794633, 3.0841272804773183, 1.9919957187095272, 1.5748800301635502, 2.0010284084234113, 2.253964310785545, 2.991535445237471, 4.029106591042163, 3.5587623505725183, 3.1329865527693084, 3.4202

In [5]:
E,V = cmp.energy_rdkit_conformers(m2,cids)
E = hdt.hatokcal*(E-E.min())
efile = open('/nh/nest/u/jsmith/scratch/Research/confs_test/fentanyl.xyz')
idx = np.argsort(E)
print(E,idx)

AttributeError: 'ensemblemolecule' object has no attribute 'compute_mean_energies'

In [ ]:
X,S = pya.__convert_rdkitconfs_to_nparr__(m2)
Xs = X[idx]
Es = E[idx]
Vs = V[idx]

In [ ]:
comments = [str(es)+','+str(vs) for es,vs in zip(Es,Vs)]
hdt.writexyzfilewc('/nh/nest/u/jsmith/scratch/Research/confs_test/Letermovir.xyz',Xs,S,comments)